In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.model_selection import train_test_split
import cv2

In [51]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('uint8')   
x_test = x_test.astype('uint8')

x_train.shape

x_train = x_train[:2000]
x_test = x_test[:2000]
y_train = y_train[:2000]
y_test = y_test[:2000]

In [52]:
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)
x_train = x_train.astype('float32')   
x_test = x_test.astype('float32')

In [53]:
kVals = [1,5,10]
accuracies = []
for k in kVals:
    # train the classifier with the current value of `k`
    model = KNeighborsClassifier(n_neighbors=k)

    model.fit(x_train, y_train)

    # evaluate the model and print the accuracies list
    score = model.score(x_test, y_test)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))
    accuracies.append(score)

k=1, accuracy=86.75%
k=5, accuracy=85.90%
k=10, accuracy=85.65%


In [32]:
kVals = [1,5,10]

for k in kVals:
    count = 0
    for i in range(len(x_test)):
        x = x_test[i]
        
        distances = [np.linalg.norm(x - data_point) for data_point in x_train]

        k_neighbors_indices = np.argsort(distances)[:k]
        k_neighbor_labels = [y_train[i] for i in k_neighbors_indices]
        
        # Return the most common class label
        most_common = np.bincount(k_neighbor_labels).argmax()
        if most_common == y_test[i]:
            count += 1
    score = count / len(x_test)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))



k=1, accuracy=82.80%


KeyboardInterrupt: 

In [54]:
from sklearn.tree import DecisionTreeClassifier

In [55]:
# Train a Decision Tree on the data (Decision tree uses IG for the splits)
tree = DecisionTreeClassifier(random_state=0)
tree.fit(x_train, y_train)
importances = tree.feature_importances_
indices = np.argsort(importances)[::-1]
indices[0]

101

In [56]:
N = 100  # Change this to select number of features based on IG
selected_features = indices[:N]
print(selected_features)
# Apply the selection to training and test data
x_train_selected = x_train[:, selected_features]
x_test_selected = x_test[:, selected_features]

x_train[1][347]
# x_train_selected[1][0]

[101 455 461 401 211 377 263 178 541 271 376 155 378 567 408 318 487 296
 653 298  97 273 319 381 566 656 657 626 438 291 463 295 404 468 316 518
 153 323 458 513 326  95 349 488 431 521 351 539 267 380 188 405 156 190
 191 352 372 429 426 407 570 602 356 632 459 128 234 350 315 290 540 467
 346 265 207 460 182 294 180 597 486 519 576 268 216 300 375 344 500 208
 692 515 382 628 573 466 325 210 516 347]


71.0

In [57]:
kVals = [1, 5, 10, 20, 30]
accuracies = []

for k in kVals:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(x_train_selected, y_train)
    score = model.score(x_test_selected, y_test)
    print(f"k={k}, accuracy={score * 100:.2f}%")
    accuracies.append(score)

k=1, accuracy=83.95%
k=5, accuracy=83.75%
k=10, accuracy=82.35%
k=20, accuracy=80.05%
k=30, accuracy=77.50%


In [58]:
kVals = [5,10,20,30,50,100]
for k in kVals:
    count = 0
    for i in range(len(x_test_selected)):
        x = x_test_selected[i]

        distance_matrix = np.argsort(abs(x - x_train_selected), axis=0)
        k_neighbor_labels = np.array([])
        k_star = k
        for l in range(len(x)):
            if k_star <= 0: break
            # print(l)
            # print('...')
            # print(distance_matrix[:, l][:k_star])
            distances_along_axis = np.array(distance_matrix[:,l][:k_star])

            k_neighbor_labels = np.concatenate([k_neighbor_labels,distances_along_axis])
            k_star -= 1


        # k_neighbors_indices = np.argsort(distances)[:k]
        # k_neighbor_labels = [y_train[i] for i in k_neighbors_indices]
        
        # Return the most common class label
        # print(k_neighbor_labels)
        
        labels = y_train[np.array(k_neighbor_labels).flatten().astype(int)]

        most_common = np.bincount(labels).argmax()
        if most_common == y_test[i]:
            count += 1
    score = count / len(x_test_selected)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))

k=5, accuracy=15.65%
k=10, accuracy=22.65%


KeyboardInterrupt: 

In [59]:
kVals = [5,10,20,30,50,100]
for k in kVals:
    count = 0
    for i in range(len(x_test)):
        x = x_test[i]
        k_neighbor_labels = np.array([])
        k_star = k
        for l in selected_features:
            # distances = [np.linalg.norm(x - data_point) for data_point in x_train]
            imp_feature_neighbours = np.array([l, l-1, l+1, l+28, l-28])
            distances = np.linalg.norm(x[imp_feature_neighbours] - x_train[:,imp_feature_neighbours], axis=1)
            distances_along_axis = np.argsort(distances)[:k_star]
            k_neighbor_labels = np.concatenate([k_neighbor_labels,distances_along_axis])
            k_star -= 1
        
        labels = y_train[np.array(k_neighbor_labels).flatten().astype(int)]

        most_common = np.bincount(labels).argmax()
        if most_common == y_test[i]:
            count += 1
    score = count / len(x_test_selected)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))

k=5, accuracy=10.35%
k=10, accuracy=10.30%
k=20, accuracy=10.30%
k=30, accuracy=10.65%
k=50, accuracy=20.45%
k=100, accuracy=69.90%


array([6731.7515, 4297.405 , 2842.059 , 6553.6353, 6194.0645],
      dtype=float32)